In [1]:
import pandas as pd
import os

# Отримати список файлів у директоріях
folder_path_idle = 'C:/Users/IJHEA/Documents/Python/HWDS-Rizun/homework/data/idle'
folder_path_running = 'C:/Users/IJHEA/Documents/Python/HWDS-Rizun/homework/data/running'
folder_path_stairs = 'C:/Users/IJHEA/Documents/Python/HWDS-Rizun/homework/data/stairs'
folder_path_walking = 'C:/Users/IJHEA/Documents/Python/HWDS-Rizun/homework/data/walking'

file_list_idle = [os.path.join(folder_path_idle, file) for file in os.listdir(folder_path_idle) if file.endswith('.csv')]
file_list_running = [os.path.join(folder_path_running, file) for file in os.listdir(folder_path_running) if file.endswith('.csv')]
file_list_stairs = [os.path.join(folder_path_stairs, file) for file in os.listdir(folder_path_stairs) if file.endswith('.csv')]
file_list_walking = [os.path.join(folder_path_walking, file) for file in os.listdir(folder_path_walking) if file.endswith('.csv')]

# Функція для завантаження та додавання колонки активності
def load_and_label(file_list, activity_label):
    dataframes = []
    for file in file_list:
        dataframe = pd.read_csv(file)
        dataframe['activity'] = activity_label
        dataframes.append(dataframe)
    return pd.concat(dataframes, ignore_index=True)

# Завантаження та додавання колонки активності для кожного типу активності
combined_dataframe_idle = load_and_label(file_list_idle, 0)
combined_dataframe_running = load_and_label(file_list_running, 1)
combined_dataframe_stairs = load_and_label(file_list_stairs, 2)
combined_dataframe_walking = load_and_label(file_list_walking, 3)

# Об'єднання датасетів вертикально (за рядком)
combined_dataframe_activity = pd.concat([
    combined_dataframe_idle,
    combined_dataframe_running,
    combined_dataframe_stairs,
    combined_dataframe_walking
], ignore_index=True)

# Збереження об'єднаного датасету у файл CSV
combined_dataframe_activity.to_csv('combined_dataframe_activity.csv', index=False)

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Завантаження даних
data = pd.read_csv('combined_dataframe_activity.csv')

# Розділення даних на ознаки та мітки
X = data.drop(columns=['activity'])
y = data['activity']

# Розділення даних на тренувальний та тестовий набори
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Тренування моделі
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Прогнозування на тестових даних
y_pred = model.predict(X_test)

# Оцінка моделі
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Приклад прогнозування для нових даних
new_data = pd.DataFrame({
    'accelerometer_X': [0.1, 0.2, 0.3],
    'accelerometer_Y': [0.4, 0.5, 0.6],
    'accelerometer_Z': [0.7, 0.8, 0.9]
})

predicted_activity = model.predict(new_data)
print("Predicted Activities:", predicted_activity)

Accuracy: 0.9995873310636542
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6203
           1       1.00      1.00      1.00     20485
           2       1.00      0.99      0.99      1022
           3       1.00      1.00      1.00     11062

    accuracy                           1.00     38772
   macro avg       1.00      1.00      1.00     38772
weighted avg       1.00      1.00      1.00     38772

Predicted Activities: [1 1 1]


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler

# Функція для аугментації даних
def augment_data(data, n_augmentations=3):
    augmented_data = []
    for _ in range(n_augmentations):
        noise = np.random.normal(0, 0.1, data.shape)
        augmented_data.append(data + noise)
    return pd.concat(augmented_data, ignore_index=True)

# Зчитування датасету
combined_dataframe_activity = pd.read_csv('combined_dataframe_activity.csv')

# Перевірка наявності NaN у колонці activity
print(combined_dataframe_activity['activity'].isna().sum())

# Видалення рядків з NaN у колонці activity
combined_dataframe_activity = combined_dataframe_activity.dropna(subset=['activity'])

# Аугментація даних
augmented_dataframe_activity = augment_data(combined_dataframe_activity)

# Перевірка наявності NaN після аугментації
print(augmented_dataframe_activity.isna().sum())

# Розрахунок часових ознак (time domain features)
def calculate_time_domain_features(data):
    features = ['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']
    time_domain_features = data[features].agg(['mean', 'std', 'max', 'min', 'median']).T
    
    # Заміна NaN значень на нулі
    time_domain_features = time_domain_features.fillna(0)
    
    time_domain_features['activity'] = data['activity']
    return time_domain_features

time_domain_features = calculate_time_domain_features(augmented_dataframe_activity)

# Перевірка наявності NaN після розрахунку часових ознак
print(time_domain_features.isna().sum())

# Розділення даних на ознаки та мітки
X = time_domain_features.drop(columns=['activity'])
y = time_domain_features['activity']

# Розділення даних на тренувальний та тестовий набори
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Нормалізація даних
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Тренування моделі SVM
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)

# Прогнозування на тестових даних
y_pred = svm_model.predict(X_test)

# Оцінка моделі
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

0
accelerometer_X    0
accelerometer_Y    0
accelerometer_Z    0
activity           0
dtype: int64
mean        0
std         0
max         0
min         0
median      0
activity    3
dtype: int64


ValueError: Input y contains NaN.